In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
import pickle

In [ ]:
hv.extension("bokeh")

# Load data

In [ ]:
with open("190311photobleaching.pickle", "rb") as f:
    d = pickle.load(f)

In [ ]:
list(d.keys())

In [ ]:
measurements, regionprops, labels, img = d[
    "/n/scratch2/jqs1/fidelity/190311/190311_mGFPmut2_100ms_laser100pct_006.nd2"
]

In [ ]:
# regionprops.reset_index(inplace=True)
regionprops.head()

In [ ]:
%%output size=150
#%%opts Image {+axiswise}
hv.Image(img / img.max()).options(cmap="gray") + hv.Image(labels != 0).options(
    cmap="blues"
)

# Plotting

In [ ]:
list(measurements.keys())

In [ ]:
traces = []
rp_list = []
rp_df = pd.DataFrame()
for i in range(7):
    print(i)
    measurements, regionprops, labels, img = d[
        "/n/scratch2/jqs1/fidelity/190311/190311_mGFPmut2_100ms_laser100pct_00"
        + str(i)
        + ".nd2"
    ]
    traces.append(measurements["mean"][1:])
    rp_list.append(regionprops)
    print(measurements["mean"][1:].shape)
traces = np.concatenate(traces)
rp_df = pd.concat(rp_list, sort=False)
rp_df.reset_index(inplace=True)
print(traces.shape)
data = pd.DataFrame(traces)  # + np.random.normal(0,1,Gsamp.T.shape))

bins = np.arange(data[0].min() - 1, data[0].max() + 1, 50)
data["bin"] = pd.cut(data[0], bins=bins)
data = pd.concat([data, rp_df], axis=1, sort=False)
# rp_df['initial_intensity'] = df[0]
t_end = traces.shape[1]

In [ ]:
%%output size=250
# traces = measurements['mean']
idxs = np.random.permutation(traces.shape[0])
downsample = (
    10  # set to 1 to show all traces (instead of 10%); this will make your browser slow
)
curves = [
    {"x": np.arange(traces.shape[1]), "y": traces[i], "i": idxs[i]}
    for i in range(traces.shape[0] // downsample)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20", logy=True)

In [ ]:
%matplotlib inline
import pylab as plt
import pandas as pd
import numpy as np
from matplotlib import cm
from scipy.integrate import simps


def filter_df(df, prop_dict):
    processed_df = df.copy()
    for prop in prop_dict:
        processed_df = processed_df[
            (processed_df[prop] > prop_dict[prop][0])
            & (processed_df[prop] < prop_dict[prop][1])
        ]

    return processed_df


def get_stats(df, thresh):
    p = df.iloc[:, :t_end].apply(lambda x: x / x[0], axis=1)
    p["bin"] = df["bin"]
    pbar = p.groupby(p["bin"]).mean()[p.groupby(p["bin"]).size() > thresh]
    mu = (
        df.iloc[:, :t_end]
        .groupby(df["bin"])
        .mean()[df.groupby(df["bin"]).size() > thresh]
    )
    sigma2 = (
        df.iloc[:, :t_end]
        .groupby(df["bin"])
        .var(ddof=0)[df.groupby(df["bin"]).size() > thresh]
    )

    return pbar, mu, sigma2


def nu_int(pbar, mu, sigma, q=1):
    nu_dict = {}  # pd.Series()
    cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
    y = sigma2.div(mu[0].values, axis="rows")
    for name, group in pbar.groupby("bin"):
        dp = pbar.loc[name].values
        dp = dp[pbar.loc[name].values > 1 - q]
        f = y.loc[name].values[pbar.loc[name].values > 1 - q]
        nu_dict[name] = cq * simps(f, dp)

    return pd.Series(nu_dict)


def fluct_plot(pbar, mu, sigma2, thresh):
    hist_df = df.groupby(df["bin"]).size()
    hist_df = hist_df[hist_df.values > thresh]
    print(hist_df)

    plt.figure(figsize=(12, 8))

    cmap = cm.get_cmap("coolwarm")

    y = sigma2.div(mu[0].values, axis="rows")
    imax = pbar.index[-1].left
    imin = pbar.index[0].left
    q = 1 / 2
    cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
    # plt.vlines(q,0,3)

    for name, group in pbar.groupby("bin"):
        dp = pbar.loc[name].values
        dp = dp[pbar.loc[name].values > 1 - q]
        f = y.loc[name].values[pbar.loc[name].values > 1 - q]
        c = (name.left - imin) / (imax - imin)
        plt.scatter(
            1 - pbar.loc[name].values, y.loc[name].values, color=cmap(c), label=name
        )
        plt.legend()

    plt.title(
        r"$\nu = "
        + str(np.round(-cq))
        + r"\cdot \int\frac{\hat{\sigma}^2}{f_{max}}dp$ ="
        + str(np.round(nu_df.mean(), 2)),
        fontsize=20,
        pad=20,
    )
    plt.xlabel(r"$(1-\hat{p})$", fontsize=20)
    plt.ylabel(r"$\frac{\hat{\sigma}^2}{f_{max}}$", fontsize=20)


thresh = 40

prop_dict = {
    "centroid_x": [300, 1000],
    "centroid_y": [750, 1500],
    "area": [30, 150],
    0: [2000, 10000],
}
# prop_dict = {'area': [30,150],
#              0: [2000, 10000]}

df = filter_df(data, prop_dict)
pbar, mu, sigma2 = get_stats(df, thresh)


nu_df = nu_int(pbar, mu, sigma2)
print(nu_df)

fluct_plot(pbar, mu, sigma2, thresh)

In [ ]:
%%output size=250
# traces = measurements['mean']
idxs = np.random.permutation(traces.shape[0])
downsample = (
    10  # set to 1 to show all traces (instead of 10%); this will make your browser slow
)
curves = [
    {"x": np.arange(traces.shape[1]), "y": df[i], "i": idxs[i]}
    for i in range(traces.shape[0] // downsample)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20", logy=True)

In [ ]:
I0 = traces[:, 0]
plt.hist(I0, bins=30)
print(np.mean(I0), np.var(I0))

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

cmap = cm.get_cmap("coolwarm")
fig = plt.figure(figsize=(12, 8))
# ax = fig.add_subplot(111, projection='3d')
imax = pbar.index[-1].left
imin = pbar.index[0].left
for name, group in df.groupby("bin"):
    if group.shape[0] > thresh:
        c = (name.left - imin) / (imax - imin)
        #         c = (nu_dict[name] - min(nu_int))/(max(nu_int) - min(nu_int))
        z = name.left * np.ones(group.centroid_x.shape)
        plt.scatter(
            group.centroid_x, group.centroid_y, color=cmap(c), label=nu_df[name]
        )
        plt.xlim(data.centroid_x.min(), data.centroid_x.max())
        plt.ylim(data.centroid_y.min(), data.centroid_y.max())
    else:
        plt.scatter(group.centroid_x, group.centroid_y, color="k", alpha=0.1)